# Tarea5: Series de Taylor

Fecha de envío del PR inicial: **viernes 21 de abril**

Fecha de aceptación del PR: **martes 25 de abril, antes de la clase**

---

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests_taylor.jl") para cada uno de los métodos que implementen.


In [4]:
workspace()
"""
Definición de polinomios de Taylor, donde
...
"""
type Taylor{T<:Number}
    v::Array{T,1}
    n::Int
end
Taylor(v,n) = Taylor(promote(v,n)...)
function Taylor(n,v)
    a=[i=zero(Number) for i=1:n]
    for i in 1:n
        a[i]=v[i]
    end
    return Taylor(a,n)
end

Taylor{T<:Number}

In [5]:
n=7
v=[i=rand() for i=1:n]
c=[i=rand() for i=1:n]
Taylor(6,c)


LoadError: InexactError()

In [8]:
workspace()
"""
Definición de polinomios de Taylor, donde
...
"""
type Taylor{T<:Number}
    n::Int
    v::Array{T,1} 
end
Taylor(n,v) = Taylor(promote(n,v)...)
function Taylor(n)
    a=[i=zero(eltype(Number)) for i=1:n]
    for i in 1:n
        a[i]=a[i]
    end
    return Taylor(n,a)
end
import Base: +, -, *, /, ==
+(a::Taylor,b::Taylor)=Taylor(a.n,a.v+b.v)
-(a::Taylor,b::Taylor)=Taylor(a.n,a.v-b.v)

        
*(a::Taylor,b::Taylor)=Taylor(a.n,[i=a.v[i]*b.v[a.n+1-i] for i=1:a.n])
#*(a::Taylor,b::Taylor)=Taylor(a.n,[i=1/b[1](a[a.n]*c[a.n+1-i]) for i=1:a.n])
#Taylor(x0)=Taylor(x0,v)

* (generic function with 151 methods)

In [9]:
n=7
v=[i=rand() for i=1:n]
c=[i=rand() for i=1:n]
Taylor(6,c)
Taylor(length(v),v)*Taylor(length(v),c)

Taylor{Float64}(7,[0.682382,0.411359,0.112189,0.273427,0.122888,0.247852,0.0139054])

In [10]:
v=[1,2,3]
c=[1,2,3]
Taylor(length(v),v)*Taylor(length(v),c),Taylor(length(v),v)+Taylor(length(v),c)


(Taylor{Int64}(3,[3,4,3]),Taylor{Int64}(3,[2,4,6]))

\begin{eqnarray}
(f+g)_{[k]} & = & f_{[k]} + g_{[k]} ,\\
(f-g)_{[k]} & = & f_{[k]} - g_{[k]} ,\\
(f \cdot g)_{[k]} & = & \sum_{i=0}^k f_{[i]} \,g_{[k-i]} \, ,\\
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) . \\
\end{eqnarray}

In [18]:
Taylor(1)

Taylor{Int64}(1,[1,2,3])

In [ ]:
import Base: +, -, *, /, ==

# Aqui se implementan los métodos necesarios para cada función


In [ ]:
# Muestren que su código funciona con tests adecuados

using Base.Test
include("runtest_taylor.jl")

---

## Ejercicio 2

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas necesarias en `runtest_taylor.jl`.

**Para el logaritmo** (ver pág. 251 cálculo Schaum)

$$\displaystyle{L(x)=\log(g(x))= \sum_{k=0}^\infty L_{[k]} (x-x_0)^k}\hspace{2cm}g(x)=\sum_{k=0}^\infty g_{[k]} (x-x_0)^k$$

$$\frac{dL(x)}{dx}=\sum_{k=0}^\infty kL_{[k]} (x-x_0)^{k-1}\hspace{2cm}
\frac{dg(x)}{dx}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$
$$\frac{dL(x)}{dx}=\frac{d\log(g(x))}{dx}=\frac{1}{g(x)}\frac{dg(x)}{dx}$$

$$g(x)\frac{dL(x)}{dx}=\frac{dg(x)}{dx}$$

Entonces
$$\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\sum_{j=0}^\infty jL_{[j]} (x-x_0)^{j-1}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$



$$\left[\sum_{k=0}^\infty kL_{[k]} (x-x_0)^{k-1}\right]\left[\sum_{j=0}^\infty g_{[j]} (x-x_0)^j\right]=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\sum_{k=1}^\infty\left[\sum_{j=0}^k(k-j)L_{[k-j]}g_{[j]} \right] (x-x_0)^{k-1}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\sum_{k=1}^\infty\left[\sum_{j=0}^kjL_{[j]}g_{[k-j]} \right] (x-x_0)^{k-1}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$

$$\sum_{j=0}^kjL_{[j]}g_{[k-j]}=kg_{[k]}$$

$$kL_{[k]}g_{[0]}+\sum_{j=0}^{k-1}jL_{[j]}g_{[k-j]}=kg_{[k]}$$

$$L_{[k]}=\frac{1}{g_{[0]}}\left[g_{[k]}-\frac{1}{k}\sum_{j=0}^{k-1}jL_{[j]}g_{[k-j]}\right]$$

**Para la función $P_\alpha(x)=(g(x))^\alpha$** 

$$\displaystyle{P_\alpha(x)=(g(x))^\alpha= \sum_{k=0}^\infty {P_\alpha}_{[k]} (x-x_0)^k}\hspace{2cm}g(x)=\sum_{k=0}^\infty g_{[k]} (x-x_0)^k$$

$$\frac{dP_\alpha(x)}{dx}=\sum_{k=0}^\infty k{P_\alpha}_{[k]} (x-x_0)^{k-1}\hspace{2cm}
\frac{dg(x)}{dx}=\sum_{k=0}^\infty kg_{[k]} (x-x_0)^{k-1}$$
$$\frac{d(g(x))^\alpha}{dx}=\alpha (g(x))^{\alpha-1}\frac{dg(x)}{dx}=\alpha \frac{(g(x))^{\alpha}}{g(x)}\frac{dg(x)}{dx}$$

Por una parte
$$\frac{d(g(x))^\alpha}{dx}=\alpha \frac{\sum_{k=0}^\infty {P_\alpha}_{[k]} (x-x_0)^k}
{\sum_{j=0}^\infty g_{[j]} (x-x_0)^j}\sum_{i=0}^\infty ig_{[i]} (x-x_0)^{i-1}$$

$$\sum_{k=0}^\infty k{P_\alpha}_{[k]} (x-x_0)^{k-1}=\alpha \frac{\sum_{k=0}^\infty {P_\alpha}_{[k]} (x-x_0)^k}
{\sum_{j=0}^\infty g_{[j]} (x-x_0)^j}\sum_{i=0}^\infty ig_{[i]} (x-x_0)^{i-1}$$

**Para la función $S(x)=\sin(g(x))$ y $C(x) = \cos(g(x))$** 

$$S(x)=sin(g(x))= \sum_{k=0}^\infty {S}_{[k]} (x-x_0)^k\hspace{2cm}C(x) = \cos(g(x))=\sum_{k=0}^\infty C_{[k]} (x-x_0)^k$$

$$\frac{dS(x)}{dx}=\sum_{k=0}^\infty k{S}_{[k]} (x-x_0)^{k-1}\hspace{2cm}
\frac{dC(x)}{dx}=\sum_{k=0}^\infty k{C}_{[k]} (x-x_0)^{k-1}$$
$$\frac{dS(x)}{dx}=\cos(g(x))\frac{dg(x)}{dx}=C(x)\frac{dg(x)}{dx}\hspace{2cm}
\frac{dC(x)}{dx}=-\sin(g(x))\frac{dg(x)}{dx}=-S(x)\frac{dg(x)}{dx}$$

Igualando las últimas 4 expresiones
$$\frac{dS(x)}{dx}=\sum_{k=0}^\infty k{S}_{[k]} (x-x_0)^{k-1}=
\sum_{k=0}^\infty C_{[k]} (x-x_0)^k\sum_{j=0}^\infty jg_{[j]} (x-x_0)^{j-1}=
\sum_{k=1}^\infty \left[\sum_{j=0}^k jg_{[j]}C_{[k-j]}\right](x-x_0)^{k-1}
$$
$$\frac{dC(x)}{dx}=\sum_{k=0}^\infty k{C}_{[k]} (x-x_0)^{k-1}=
-\sum_{k=0}^\infty S_{[k]} (x-x_0)^k\sum_{j=0}^\infty jg_{[j]} (x-x_0)^{j-1}=
-\sum_{k=1}^\infty \left[\sum_{j=0}^k jg_{[j]}S_{[k-j]}\right](x-x_0)^{k-1}
$$

De aquí obtengo las dos siguientes relaciones

$$S_{[k]}=\frac{1}{k}\sum_{j=0}^k jg_{[j]}C_{[k-j]}=\frac{1}{k}\sum_{j=0}^k (k-j)g_{[k-j]}C_{[j]}\hspace{2cm}
C_{[k]}=-\frac{1}{k}\sum_{j=0}^k jg_{[j]}S_{[k-j]}=-\frac{1}{k}\sum_{j=0}^k (k-j)g_{[k-j]}S_{[j]}$$

Con las condiciones iniciales 
$$S_{[0]}=\sin(g(x_0))\hspace{2cm}C_{[0]}=\cos(g(x_0))$$

Para el caso concreto $g(x) = x$ alrededor de $x_0=0$, donde tenemos $g_{[j]} = \delta_{j,1}$, obtenemos

$$S_{[0]}=0 \hspace{2cm}C_{[0]}=1$$
$$S_{[1]}=C_{[0]}=1 \hspace{2cm}C_{[1]}=-S_{[0]}=0$$

$$S_{[k]}=\frac{1}{k}C_{[k-1]}=\frac{1(-1)}{k(k-1)}S_{[k-2]}=\frac{1(-1)(1)}{k(k-1)(k-2)}C_{[k-3]}=
\frac{1(-1)(1)(-1)}{k(k-1)(k-2)(k-3)}S_{[k-4]}=\dots$$$$
C_{[k]}=\frac{-1}{k}S_{[k-1]}=\frac{(-1)(1)}{k(k-1)}C_{[k-2]}=\frac{(-1)(1)(-1)}{k(k-1)(k-2)}S_{[k-3]}=
\frac{(-1)(1)(-1)(1)}{k(k-1)(k-2)(k-3)}C_{[k-4]}=\dots$$

$$S_{[k]}=\frac{(-1)}{k(k-1)}S_{[k-2]}=\frac{(-1)^2}{k(k-1)(k-2)(k-3)}S_{[k-4]}=
\frac{(-1)^3}{k(k-1)(k-2)(k-3)(k-4)(k-5)}S_{[k-6]}=\dots$$$$
C_{[k]}=\frac{(-1)}{k(k-1)}C_{[k-2]}=\frac{(-1)^2}{k(k-1)(k-2)(k-3)}C_{[k-4]}=
\frac{(-1)^3}{k(k-1)(k-2)(k-3)(k-4)(k-5)}C_{[k-6]}=\dots$$

$$S_{[k]}=\frac{(-1)^{j/2}(k-j)!}{k!}S_{[k-j]}\hspace{2cm}C_{[k]}=\frac{(-1)^{j/2}(k-j)!}{k!}C_{[k-j]}$$

Para que $S_{[k]}$ no sea nulo $k>2$ y debe ser un número impar $k=2n+1$. De la misma menera para que $C_{[k]}$ no sea nulo $k\geq2$ y debe ser un número par $k=2n$. Entonces

$$S_{[2n+1]}=\frac{(-1)^{j/2}((2n+1)-j)!}{(2n+1)!}S_{[(2n+1)-j]}\hspace{2cm}C_{[2n]}=\frac{(-1)^{j/2}((2n)-j)!}{(2n)!}C_{[(2n)-j]}$$

Para los coeficientes del seno si $2n+1-j=1$ $\Rightarrow j=2n$. En los coeficientes del coseno si $2n-j=0$ $\Rightarrow j=2n$. De esta manera

$$S_{[2n+1]}=\frac{(-1)^{n}}{(2n+1)!}S_{[1]}=\frac{(-1)^{n}}{(2n+1)!}C_{[0]}=\frac{(-1)^{n}}{(2n+1)!}\hspace{2cm}C_{[2n]}=\frac{(-1)^{n}}{(2n)!}C_{[0]}=\frac{(-1)^{n}}{(2n)!}$$

Por lo tanto
$$S(x)=\sin(x)=\sum_{n=0}^\infty \frac{(-1)^{n}}{(2n+1)!}x^{2n+1}\hspace{2cm}C(x) = \cos(x)=\sum_{n=0}^\infty \frac{(-1)^{n}}{(2n)!}x^{2n}$$